In [ ]:
!pip install flask-cors
from flask_cors import CORS

In [ ]:
!pip install flask pyngrok sentence-transformers faiss-gpu pandas tqdm


In [ ]:
!ngrok config add-authtoken 2neCjpEUXmbZ73g7vYGoVTHiCZX_2nAiXCF4ZApWpEMWjhE3B


In [ ]:
!pip install googletrans==4.0.0-rc1
from googletrans import Translator, LANGUAGES

# Initialize the Google Translator
translator = Translator()

In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import faiss
from sentence_transformers import SentenceTransformer
from flask import Flask, request, jsonify
from pyngrok import ngrok

# Load DataFrame
df = pd.read_csv("/kaggle/input/mydatasetfinal/final_cleaned.csv")

# Debug: Check if DataFrame loaded properly
print(f"DataFrame Loaded: {df.shape}")


# Load and encode data
def load_data():
    df['prompt_str'] = df['prompt'].apply(lambda x: json.dumps(x))
    model = SentenceTransformer('all-mpnet-base-v2')
    df['embeddings'] = df['prompt_str'].apply(lambda x: model.encode(x))
    embeddings = np.array(df['embeddings'].tolist()).astype('float32')
    
    # Initialize FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    print("FAISS index built successfully.")
    
    return df, index, model

data, index, model = load_data()

In [9]:
import json

# Function to retrieve top N matches

# Function to retrieve top N matches (with 5 parameters)

import ast

def retrieve_top_n(user_query, top_n, model, index, data):
    if user_query:
        # Detect the language of the input
        detected_language = translator.detect(user_query).lang

        if detected_language != 'en':
            # Translate to English
            translation = translator.translate(user_query, src=detected_language, dest='en')
            user_query = translation.text
            print(f"Translated Query: {user_query} (from {LANGUAGES[detected_language]})")  # Replace st.success with print
        query_embedding = model.encode(user_query).reshape(1, -1)
        distances, indices = index.search(query_embedding, top_n)
        print("distances:", distances)

        results = []
        print("indices:", indices)

    for i, idx in enumerate(indices[0]):
        try:
            print("Processing idx:", idx)
            result = data.iloc[idx]
            prompt_content = result['prompt']
            print(f"Raw JSON string: {prompt_content}")

            # Check if already a dictionary
            if isinstance(prompt_content, dict):
                prompt_dict = prompt_content
            elif isinstance(prompt_content, str):
                # First try to load as JSON
                try:
                    prompt_dict = json.loads(prompt_content)
                except json.JSONDecodeError:
                    print("Trying with ast.literal_eval...")
                    # Use ast.literal_eval as fallback for non-standard JSON
                    prompt_dict = ast.literal_eval(prompt_content)
            else:
                raise ValueError(f"Unsupported type for 'prompt': {type(prompt_content)}")
            distance = distances[0][i]
            result_metadata = {
                'Title': prompt_dict.get('Title', 'No Title'),
                'Author': prompt_dict.get('Author', 'No Author'),
                "Labels": prompt_dict.get('Labels', 'No Labels'),
                "Read Level": prompt_dict.get('Read Level', 'No Read level'),
                'Hyperlink': prompt_dict.get('Hyperlink', 'No Hyperlink'),
                'Match_Percentage': f"{distance:.2f}"
            }
            print("Result metadata:", result_metadata)
            results.append(result_metadata)

        except Exception as e:
            print(f"Error processing idx {idx}: {e}")

    return results

def sanitize_json_string(json_string):
    """Sanitize the JSON string to correct common issues."""
    # Replace single quotes with double quotes
    json_string = json_string.replace("'", '"')
    # Remove extra commas before closing braces/brackets
    json_string = json_string.replace(",}", "}").replace(",]", "]")
    # Ensure proper formatting for newlines and spaces
    json_string = json_string.replace("\n", "").strip()
    return json_string


# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Route to expose retrieve_top_n to the frontend
@app.route('/retrieve', methods=['POST'])
def retrieve():
    query = request.json.get('query', '')
    print(f"Received query: {query}")

    # Call retrieve_top_n with the correct arguments
    result = retrieve_top_n(query, 5, model, index, data)
    print(f"Returning result: {result}")
    return jsonify(result)

# Ngrok public URL configuration
public_url = ngrok.connect(5001)
print(f"Access your app at: {public_url}")

# Run the Flask app
if __name__ == "__main__":
    app.run(port=5001)
    